In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

from lightgbm import LGBMClassifier
from lightgbm import plot_importance, plot_metric, early_stopping

In [ ]:
questions = pd.read_csv('../input/feature-engineering-datasets/questions_fe.csv')
question_stats = pd.read_csv('../input/feature-engineering-datasets/questions_stats_fe.csv')

In [ ]:
#TO DO:
#
#MODIFICARE AGGIORNAMENTO USER PERFORMANCE USANDO DATATABLE (FARE COUNT E SUM SUI CAMPI DI INTERESSE)

#given the dataset, I retrieve the cumulative history of students
def get_last_performance( data,last_record=pd.DataFrame()):
    last_record.index= -1 - last_record.index
    df = data[data['content_type_id']==0].merge(questions[['question_id','part','kmean_cluster']], left_on='content_id', right_on = 'question_id', how = 'left')
    df = df[['timestamp','user_id','answered_correctly','part','row_id','kmean_cluster']]
    df['part_answer'] = 'part_'+ df.part.astype(str)  + '_' + df.answered_correctly.astype(str)
    df['cluster_answer'] = 'cluster_'+ df.kmean_cluster.astype(str)  + '_' + df.answered_correctly.astype(str)
    
    user_performance = df[['user_id','row_id']].merge(
        pd.get_dummies(df['part_answer']),
        left_index = True, 
        right_index = True).merge(
        pd.get_dummies(df['cluster_answer']),
        left_index = True, 
        right_index = True).set_index('row_id')
    try:
        user_performance = user_performance.append(last_record.drop(columns=['timestamp'],inplace=False))
    except:
            None
    user_performance = user_performance.groupby('user_id').cumsum().merge(
        df[['user_id','row_id','timestamp']].set_index('row_id'), left_index = True, right_index = True)
    user_performance_part_last = last_record.append(user_performance).groupby('user_id',as_index=False).max().fillna(0)
    
    return user_performance, user_performance_part_last

In [ ]:
get_last_performance(pd.read_csv('../input/riiid-test-answer-prediction/train.csv',nrows=100))

In [ ]:
target_col = ['answered_correctly']
feature_col = [
 'timestamp',
 'prior_question_elapsed_time',
    'prior_question_had_explanation',
    'part',
    'kmean_cluster',
    'content_type_id',
 'part_1_avg',
 #'part_1_count',
 'part_2_avg',
 #'part_2_count',
 'part_3_avg',
 #'part_3_count',
 'part_4_avg',
 #'part_4_count',
 'part_5_avg',
 #'part_5_count',
 'part_6_avg',
 #'part_6_count',
 'part_7_avg',
 #'part_7_count',
 'cluster_0_avg',
 #'cluster_0_count',
 'cluster_1_avg',
 #'cluster_1_count',
 'cluster_2_avg',
 #'cluster_2_count',
 'cluster_3_avg',
 #'cluster_3_count',
 'cluster_4_avg',
 #'cluster_4_count',
    'cluster_5_avg',
 #'cluster_5_count',
 'cluster_6_avg',
 #'cluster_6_count',
 'bundle_mean_answered_correctly',
 'bundle_std_answered_correctly',
 'bundle_perc_students',
 #'bundle_times',
 'part_mean_answered_correctly',
 'part_std_answered_correctly',
 'part_perc_students',
 #'part_times',
 'cluster_mean_answered_correctly',
 'cluster_std_answered_correctly',
 'cluster_perc_students'
# 'cluster_times'
]

In [ ]:
lgbm = LGBMClassifier(
    num_leaves = 21,
    max_depth = 5,
    n_estimators = 100,
    min_child_samples = 1000, 
    subsample=0.7, 
    n_jobs= -1,
    min_data_in_leaf = 100,
    is_higher_better = True,
    first_metric_only = True,
    feature_fraction = 0.5,
    lambda_l1 = 0.1,
    learning_rate = 0.1
)

In [ ]:
import gc

iter_df =  pd.read_csv('../input/riiid-test-answer-prediction/train.csv',chunksize = 1000000)

user_last_performance = pd.DataFrame()
debug = []
score_old = 0

for i,df in enumerate(iter_df):
    #print('Running iteration: {}'.format(i))
    gc.collect()
    user_stats = df[['row_id','user_id','content_id','content_type_id',
                                                  'timestamp','answered_correctly',
                                                  'prior_question_elapsed_time',
                                                  'prior_question_had_explanation']]
    user_performance, user_last_performance = get_last_performance(df, user_last_performance)
    print(user_performance)
    data = user_stats.merge(
        user_performance.reset_index().rename(columns={'index':'row_id'}),
        on =['row_id','user_id','timestamp'], how = 'inner'
    ).merge(
        question_stats, left_on = 'content_id', right_on = 'question_id'
    ).merge(
        questions, left_on = 'question_id', right_on = 'question_id'
     ).set_index('row_id')
    
    #Rescaling some features
    data = data.astype({'prior_question_had_explanation':'bool'})
    data['prior_question_had_explanation'] = data['prior_question_had_explanation'].fillna(False)
    
    data.timestamp/=(365*24*60*60*100)
    data.prior_question_elapsed_time/=(60*60*24)
    data = data.fillna(-1)
    
    # Calculate stats performance on parts
    for part in questions.part.unique():
        data['part_'+str(part)+'_avg'] =  data['part_'+str(part)+'_1'].divide(data['part_'+str(part)+'_0'] + data['part_'+str(part)+'_1'])
        data['part_'+str(part)+'_count'] = data['part_'+str(part)+'_0'] + data['part_'+str(part)+'_1']
        data.drop(columns = ['part_'+str(part)+'_0', 'part_'+str(part)+'_1'],inplace = True)
    data = data.fillna(0.5)
    
    # Calculate stats performance on clusters
    
    for part in questions.kmean_cluster.unique():
        data['cluster_'+str(part)+'_avg'] =  data['cluster_'+str(part)+'_1'].divide(data['cluster_'+str(part)+'_0'] + data['cluster_'+str(part)+'_1'])
        data['cluster_'+str(part)+'_count'] = data['cluster_'+str(part)+'_0'] + data['cluster_'+str(part)+'_1']
        data.drop(columns = ['cluster_'+str(part)+'_0', 'cluster_'+str(part)+'_1'],inplace = True)
    data = data.fillna(0.5)
    
    # Model Training & Validation in Loop
    
    X_train,X_test,Y_train, Y_test = train_test_split(data[feature_col], data[target_col], test_size = 0.2, shuffle = False)
    
    
    
    if i == 0:
        lgbm.fit(X_train, Y_train,callbacks = [early_stopping])
    score_train = roc_auc_score(Y_train.values, lgbm.predict_proba(X_train)[:,1])
    if ((score_old - score_train > 0.15) | (score_train < 0.745)) & (i%25==0):
        lgbm.fit(X_train, Y_train,callbacks = [early_stopping])
        score_train = roc_auc_score(Y_train.values, lgbm.predict_proba(X_train)[:,1])
    score_test = roc_auc_score(Y_test.values, lgbm.predict_proba(X_test)[:,1])
    #if i%10==0:
    print('\nRun: {}'.format(i))
    print('Score Train: {}'.format(score_train))
    print('Score Test: {}'.format(score_test))

    debug.append([i, len(df),len(user_stats),len(user_performance),len(user_last_performance),score_train, score_test])
    score_old = score_train
    
    del user_performance, score_train, score_test, data, X_train, X_test, Y_train, Y_test
    #print('\nClosed iteration: {}'.format(i))
    #if i == 10:
    #    break

debug_df = pd.DataFrame(debug, columns = [
    'run', 'len_chunk', 'len_user_stats', 'len_performance','num_users','score_train','score_test'
])

debug_df[['score_train','score_test']].plot.line()    

plot_importance(lgbm, figsize  = (18,10))

In [ ]:
# salvataggio del modello lgbm
lgbm.booster_.save_model('./model.txt')
# salvataggio user_last_performance
user_last_performance.to_csv('./user_stats.csv', index = False)

In [ ]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null

import lightgbm as lgb
import datatable as dt
from datatable import f, by
import numpy as np 

In [ ]:
user_last_performance = dt.fread('./user_stats.csv')
question_stats = dt.fread('../input/feature-engineering-datasets/questions_stats_fe.csv')
questions = dt.fread('../input/feature-engineering-datasets/questions_fe.csv')
questions.names={'question_id':'content_id'}
questions.key='content_id'
question_stats.names={'question_id':'content_id'}
question_stats.key='content_id'
user_last_performance.key='user_id'

In [ ]:
aggregation_dict = {}
aggregation_perf_dict = {}
for part in dt.unique(questions[:,'part']).to_list()[0]:
    for i in (0,1):
        aggregation_dict['part_'+str(part)+'_'+str(i)] = dt.sum((f.answered_correctly==0)&(f.part==part))
        aggregation_perf_dict['part_'+str(part)+'_'+str(i)] = dt.sum(f['part_'+str(part)+'_'+str(i)])
for part in dt.unique(questions[:,'kmean_cluster']).to_list()[0]:
    for i in (0,1):
        aggregation_dict['cluster_'+str(part)+'_'+str(i)] = dt.sum((f.answered_correctly==0)&(f.part==part))
        aggregation_perf_dict['cluster_'+str(part)+'_'+str(i)] = dt.sum(f['cluster_'+str(part)+'_'+str(i)])
aggregation_dict['timestamp'] = dt.max(f.timestamp)
aggregation_perf_dict['timestamp'] = dt.max(f.timestamp)

In [ ]:
#MODIFICARE DATASET CON DATATABLE MIGLIORANDO L'EFFICIENZA

def enrich_dataset(dataset, last_record):
    
    # RICEVO IL DATASET SU CUI APPLICARE I RISULTATI
    
    #AGGIUNGO GLI ATTRIBUTI PRECALCOLATI PER TUTENTE
    
    #AGGIUNGO GLI ATTRIBUTI DAI VARI DATASETS
    
    #RESTITUISCO DATASET ARRICCHITO
    
    data = dt.Frame(
        dataset[['content_type_id','row_id','user_id','content_id','timestamp'
                 ,'prior_question_elapsed_time','prior_question_had_explanation']].fillna(-60*60*24)
    )
    content_type_id,row_id, user_id, content_id, timestamp, prior_question_elapsed_time,prior_question_had_explanation = data.export_names()
    data = data[:,{'row_id':row_id, 
                   'user_id':user_id, 
                   'content_id':content_id, 
                   'timestamp':timestamp/(365*24*60*60*100), 
                   'prior_question_elapsed_time':prior_question_elapsed_time/(60*60*24),
                  'content_type_id':content_type_id,
                  'prior_question_had_explanation':prior_question_had_explanation}]
    data = data[:,:,dt.join(question_stats)]
    X = user_last_performance[:, dt.f[:].remove(dt.f['timestamp'])]
    X.key = 'user_id'
    data = data[:,:,dt.join(X)]
    
    del X

    for part in dt.unique(questions[:,'part']).to_list()[0]:
        col1 = 'part_'+str(part)+'_avg'
        col2 = 'part_'+str(part)+'_count'
        part_1 = 'part_'+str(part)+'_1'
        part_0 = 'part_'+str(part)+'_0'
        data[:,col1] = data[:,dt.f[part_1]/(dt.f[part_0]+dt.f[part_1])]
        data[:,col2] = data[:,dt.f[part_0] + dt.f[part_1]]
        del data[:, part_1]
        del data[:, part_0]
    for part in dt.unique(questions[:,'kmean_cluster']).to_list()[0]:
        col1 = 'cluster_'+str(part)+'_avg'
        col2 = 'cluster_'+str(part)+'_count'
        part_1 = 'cluster_'+str(part)+'_1'
        part_0 = 'cluster_'+str(part)+'_0'
        data[:,col1] = data[:,dt.f[part_1]/(dt.f[part_0]+dt.f[part_1])]
        data[:,col2] = data[:,dt.f[part_0] + dt.f[part_1]]
        del data[:, part_1]
        del data[:, part_0]
    return data.to_pandas().fillna(0.5)

In [ ]:
def get_last_performance(last_df, user_last):
    from datatable import f, by 
    data = dt.Frame(last_df.loc[last_df['content_type_id']==0,['timestamp','user_id','row_id','answered_correctly','content_id']])
    data.key = 'row_id'
    X = data[:,:,dt.join(questions)]
    del X[:,['content_id','bundle_id','tags']]
    X.key = 'row_id'
    data = X[:,aggregation_dict,by('user_id')]
    data.rbind(user_last)
    X = data[:,aggregation_perf_dict,by('user_id')]    
    return X

In [ ]:
d = {'row_id':'int64','answered_correctly':'float64'}

import gc

last_df = pd.DataFrame()

for (test_df,current_prediction_df) in iter_test:
    
    gc.collect()
    
    if last_df.shape[0]>0:
        last_df['answered_correctly'] = np.array(eval(test_df['prior_group_answers_correct'].iloc[0]))
        #%time
        user_last_performance = get_last_performance(last_df, user_last_performance)
    data = enrich_dataset(test_df, user_last_performance)
    data = data.loc[data.content_type_id == 0, : ]
    
    assert len(data)==len(current_prediction_df)
    
    current_prediction_df.answered_correctly = lgbm.predict(data[feature_col].fillna(-1)).ravel()
    del data
    
    env.predict(current_prediction_df.fillna(0.5).astype(d))
    last_df = test_df.copy(deep=True)